In [1]:
import sqlite3
import utils
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier
from sklearn import ensemble
from scipy.sparse import coo_matrix
from sklearn import svm

import xgboost
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import numpy as np
import torch

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
import keras.backend as K

In [3]:
def w2v_average_wiki():
    conn=sqlite3.connect('songs.db')
    cur=conn.cursor()
    example_text=cur.execute("select Lyrics from song where Lyrics is not null").fetchall()
    labels=[i[0] for i in cur.execute("select Economic from song where Lyrics is not null").fetchall()]
    songs=[]
    for i in example_text:
        songs.append(utils.tokenize_word(utils.clean_text(i[0])))
    songs=utils.w2v_average_wiki(songs,'conceptnet-numberbatch-17-06-300')
    return np.array(songs),np.array(labels)

In [4]:
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

In [5]:
X,y=w2v_average_wiki()

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\numpy\lib\function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.5)

In [7]:
def dummy(X_train,X_test,y_train,y_test):
    dummy = DummyClassifier()
    dummy.fit(X_train, y_train)
    y_pred=dummy.predict(X_test)
    return f1_score(y_test,y_pred)

In [8]:
print(dummy(X_train,X_test,y_train,y_test))

0.4489655172413793


In [9]:
def logistics_sentiment():
    conn=sqlite3.connect('SONGS.db')
    data=pd.read_sql("select Economic,neg,pos,neu,comp from song",conn).dropna()
    X=data[['neg','pos','neu','comp']]
    y=data.Economic
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
    logreg = LogisticRegression()
    logreg.fit(X_train,y_train)
    y_pred=logreg.predict(X_test)
    return f1_score(y_test,y_pred)

In [10]:
print(logistics_sentiment())

0.0


C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
def logistics(X_train,X_test,y_train,y_test):
    logreg = LogisticRegression()
    logreg.fit(X_train,y_train)
    y_pred=logreg.predict(X_test)
    return f1_score(y_test,y_pred)

In [12]:
print(logistics(X_train,X_test,y_train,y_test))

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Expected 2D array, got 1D array instead:
array=[nan nan nan ... nan nan nan].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [13]:
def naive_bayes(X_train,X_test,y_train,y_test):
    nb=GaussianNB()
    nb.fit(X_train, y_train)
    y_pred=nb.predict(X_test)
    return f1_score(y_test,y_pred)

In [14]:
print(naive_bayes(X_train,X_test,y_train,y_test))

ValueError: Expected 2D array, got 1D array instead:
array=[nan nan nan ... nan nan nan].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [15]:
def support_vector(X_train,X_test,y_train,y_test):
    model = svm.SVC()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [16]:
print(support_vector(X_train,X_test,y_train,y_test))

ValueError: Expected 2D array, got 1D array instead:
array=[nan nan nan ... nan nan nan].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
def knn(X_train,X_test,y_train,y_test):
    model = KNeighborsClassifier(n_neighbors=5) 
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [ ]:
print(knn(X_train,X_test,y_train,y_test))

In [ ]:
def decision_tree(X_train,X_test,y_train,y_test):
    model = DecisionTreeClassifier(random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [ ]:
print(decision_tree(X_train,X_test,y_train,y_test))

In [ ]:
def random_forest(X_train,X_test,y_train,y_test):
    model=ensemble.RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [ ]:
print(random_forest(X_train,X_test,y_train,y_test))

In [ ]:
def xgb(X_train,X_test,y_train,y_test):
    X_train=coo_matrix(X_train).tocsc()
    X_test=coo_matrix(X_test).tocsc()
    model=xgboost.XGBClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [ ]:
print(xgb(X_train,X_test,y_train,y_test))

In [ ]:
def simple_nn(X_train,X_test,y_train,y_test):

    def create_model_architecture(input_size):
        input_layer = layers.Input(shape=(input_size,),)
        hidden_layer = layers.Dense(200, activation="relu")(input_layer)
        output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)
        classifier = models.Model(inputs = input_layer, outputs = output_layer)
        classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
        return classifier

    model=create_model_architecture(len(X_train[0]))
    model.fit(X_train, y_train,epochs=100)
    y_pred = [int(round(i[0])) for i in model.predict(X_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [ ]:
print(simple_nn(X_train,X_test,y_train,y_test))

In [ ]:
limit_mem()